In [1]:
import sys
import os
sys.path.append(os.path.abspath('../version_3'))
from embedding import update_embeddings, get_embeddings, DatabaseConnection

update_embeddings()
embeds = get_embeddings()

Connected to MySQL server
MySQL connection is closed


In [2]:
from openai import OpenAI

client = OpenAI()
EMBEDDING_MODEL = "text-embedding-3-small"

def create_embedding(data):

    response = client.embeddings.create(model=EMBEDDING_MODEL, input=[data])
    embedding = [e.embedding for e in response.data][0]

    return embedding

In [3]:
from scipy import spatial

def get_top_N_related_tables(query, N=5):
    query_embed = create_embedding(query)
    relatedness_fn = lambda x, y: 1 - spatial.distance.cosine(x, y)
    score = []

    for table, embedding in embeds.items():
        score.append((table, relatedness_fn(query_embed, embedding)))

    score.sort(key=lambda x: x[1], reverse=True)
    return score[:N]

In [57]:
def generate_selection_prompt(candidates, query):
    
    prompt = 'Here are the tables available:\n'

    for table, _ in candidates:
        schema = db_conn.describe_table(*table.split('.'))
        table_structure = str(pd.DataFrame(schema, columns=["Field", "Type", "Null", "Key", "Default", "Extra"]))
        prompt += f'{table}:\n{table_structure}\n\n'

    prompt += "Please find which tables are relevant to solve queries.\n\n"

    prompt += f"Query: In the {table.split('.')[1]} table of {table.split('.')[0]} database, is the {schema[0][0]} column a primary key?\n"
    prompt += f"Relevant Tables: ['{table}']\n"
    prompt += f"Query: In the {table.split('.')[1]} table of {table.split('.')[0]} database, Are there multiple values of {schema[0][0]} which are same?\n"
    prompt += f"Relevant Tables: ['{table}']\n"
    prompt += f"Query: {query}\n"
    prompt += f"Relevant Tables: "

    return prompt

In [ ]:
import pandas as pd

def select_relevant_tables(query):
    candidates = get_top_N_related_tables(query)
    prompt = generate_selection_prompt(candidates, query)

    


    

In [58]:
query = 'How many variant upgrade trackers are in a PENDING state?'
candidates = get_top_N_related_tables(query)
prompt = generate_selection_prompt(candidates, query)

In [60]:
print(prompt)

Here are the tables available:
grimlock_dev_db.variant_upgrade_tracker_seq:
      Field    Type Null Key Default Extra
0  next_val  bigint  YES        None      

grimlock_dev_db.variant_upgrade_tracker:
                      Field                                 Type Null  Key  \
0                        id                               bigint   NO  PRI   
1     account_creation_date                         varchar(255)  YES        
2                awb_number                         varchar(255)  YES        
3        card_creation_date                         varchar(255)  YES        
4          card_holder_type                         varchar(255)  YES        
5            card_last_four                         varchar(255)  YES        
6             card_old_logo                         varchar(255)  YES        
7                 card_type                         varchar(255)  YES        
8                created_on                          datetime(6)  YES        
9    cust_update